In [2]:
import pandas as pd
import re
import os ,sys
from typing import List, Dict 

In [3]:
def read_xlsx(file_path:str, index = False)->pd.DataFrame:
    df = pd.read_excel(file_path, header=0,index_col = None )
    print(df.info())
    return df




def record_to_json(path:str):
    """
    Converts a given record file in .xlsx format to JSON format.

    Parameters:
    - path (str): The path to the record file.

    Returns:
    None
    """
    json_path = os.path.splitext(path)[0]+".json"
    data = read_xlsx(path).to_json(json_path,orient="records", indent=4) # here indent=4 makes it prettier
    print(f"Data saved to {json_path}")


In [4]:
def image_url_parser(recipe_card:str)->str:

    pattern = r'https[^"]+\.jpg'
    print(recipe_card)
    match = re.findall(pattern, recipe_card)
    if match:
        return match[0]

    return None

In [5]:
file_path = r"C:\Users\ayhan\Desktop\ChefApp\artifacts\recipes\new_data\chineese\chineese_appetizers.xlsx"
sub_category = file_path.split("\\")[-1].split(".")[0]
sub_category

'chineese_appetizers'

In [329]:
import os
import re
import pandas as pd
from typing import List

class Preprocessor:
    def __init__(self) -> None:
        print("preprocessor initialized")
        self.df = pd.DataFrame([])  # initialize an empty dataframe
        self.file_path = None

    def read_xlsx(self, file_path: str, features: List, index_col=False) -> None:
        try:
            sub_category = file_path.split("\\")[-1].split(".")[0]
            self.file_path = file_path
            df = pd.read_excel(file_path, header=0, index_col=None, usecols=features)
            df.drop_duplicates(inplace=True)  # remove duplicated rows, if any
            df.drop(df[df["recipe_directions"].isnull()].index, inplace=True)  # remove rows with null values in "recipe_directions" column
            df["sub_category"] = sub_category  # common column for all rows, derived from file name
            self.df = df
            print(df.info())
        except Exception as e:
            print("error occurred while reading xlsx file: ", e)



    def df_to_json(self, path: str = None) -> None:
        """
        Converts a given record file in .xlsx format to JSON format.
        """
        if path is None:
            path = os.path.splitext(self.file_path)[0] + ".json"

        if self.df.empty:
            raise ValueError("DataFrame is empty. Please use read_xlsx to populate the DataFrame first.")

        self.df.to_json(path, orient="records", indent=4)
        print(f"JSON saved to {path}")



    def image_url_parser(self, recipe_card: str) -> str:
        pattern = r'https[^"]+\.jpg'
        print(recipe_card)
        match = re.findall(pattern, recipe_card)
        if match:
            return match[0]
        return None


    def df_img_url_parser(self, col_name: str = "card") -> None:
        self.df["image_url_formatted"] = self.df[col_name].apply(self.image_url_parser)
        

    def recipe_tag_formatter(self) -> None:
        if "recipe_tags" not in self.df.columns:
            raise ValueError("The dataframe does not contain the 'recipe_tags' column.")

        self.df.drop(self.df[self.df["recipe_tags"].isnull()].index, inplace=True)  # remove rows with no recipe_tags
        
        self.df["recipe_tags_formatted"] = self.df["recipe_tags"].apply(lambda tags: [kv["recipe_tags"] for kv in eval(tags)])
        # Get the common sub_category value
        common_sub_category = self.df["sub_category"].iloc[0]
        
        # Append the common_sub_category to each list in the "recipe_tags_formatted" column
        self.df["recipe_tags_formatted"] = self.df["recipe_tags_formatted"].apply(lambda element: element + [common_sub_category])


    def details_table_formatter(self, table: List) -> dict:
        print(eval(table))

        table_str = eval(table)[0]["recipe_details_table"]
        print(type(table_str))
        # Split the details string based on both newline and ":"
        details_list = re.split(r'\n|:', table_str)

        # Remove empty strings and strip any leading/trailing whitespace
        table_detail_list = [detail.strip() for detail in details_list if detail.strip()]

        print(table_detail_list)
        # Initialize a dictionary with specific keys and default values
        table_details_dict = {
            "Prep Time": "None",
            "Cook Time": "None",
            "Additional Time": "None",
            "Total Time": "None",
            "Servings": "None",
            "Yield": "None"
        }
        
        
        for index , item in enumerate(table_detail_list):
            print(item)
            if item in table_details_dict:
                table_details_dict[item] = table_detail_list[index+1]

        
        return table_details_dict


    def recipe_details_table_formatter(self) -> None:
        if "recipe_details_table" not in self.df.columns:
            raise ValueError("The dataframe does not contain the 'recipe_details_table' column.")

        #self.df.drop(self.df[~self.df["recipe_details_table"].str.contains("Time")].index, inplace=True)  # remove rows not containing any time detail.

        #print(eval(self.df["recipe_details_table"].iloc[0])) List[dict[str, str]]
        self.df["recipe_details_formatted"] = self.df["recipe_details_table"].apply(lambda tags: [kv["recipe_details_table"] for kv in eval(tags)])
        # Convert "recipe_details_table" into a dictionary of key-value pairs
        self.df["recipe_details_formatted"] = self.df["recipe_details_table"].apply(lambda tags: self.details_table_formatter(tags))




    def ingredient_list_formatter(self, ingredients: List) -> List:
        """Assumes the list has 1 element, if it contains multiple elements, then just ' '.join()  them """
        ingredient_str = ingredients[0]
        ingredients_list = ingredient_str.split('\n')

        # Remove empty strings and strip any leading/trailing whitespace
        ingredients_list = [ingredient.strip() for ingredient in ingredients_list if ingredient.strip()]
        return ingredients_list


    def recipe_ingredients_formatter(self) -> None:
        if "recipe_ingredients" not in self.df.columns:
            raise ValueError("The dataframe does not contain the 'recipe_ingredients' column.")

        self.df["recipe_ingredients_formatted"] = self.df["recipe_ingredients"].apply(lambda tags: [kv["recipe_ingredients"] for kv in eval(tags)])
        self.df["recipe_ingredients_formatted"] = self.df["recipe_ingredients_formatted"].apply(lambda ingredient_list: self.ingredient_list_formatter(ingredient_list))


    def recipe_directions_formatter(self) -> None:
        if "recipe_directions" not in self.df.columns:
            raise ValueError("The dataframe does not contain the 'recipe_directions' column.")

        self.df.drop(self.df[self.df["recipe_directions"].apply(lambda element: len(element)) <= 2].index, inplace=True)
        self.df["recipe_directions_formatted"] = self.df["recipe_directions"].apply(lambda tags: [kv["recipe_directions"] for kv in eval(tags)])




    def recipe_nutrition_formatter(self, nutrition_details: List) -> dict:
        print(nutrition_details)
        nutrition_str = nutrition_details[0]
        nutrition_list = nutrition_str.split('\n')

        # Remove empty strings and strip any leading/trailing whitespace

        nutrition_list = [nutrition.strip() for nutrition in nutrition_list if nutrition.strip()]
        nutrition_info = {
            "calories": 0,
            "fat": 0,
            "carbs": 0,
            "protein": 0
        }

        for index , item in enumerate(nutrition_list):
            if item.lower() in nutrition_info:
                nutrition_info[item.lower()] = nutrition_list[index - 1]
            

        return nutrition_info


    def recipe_nutrition_details_formatter(self) -> None:
        if "recipe_nutrition_details" not in self.df.columns:
            raise ValueError("The dataframe does not contain the 'recipe_nutrition_details' column.")

        self.df.drop(self.df[self.df["recipe_nutrition_details"].apply(lambda element: len(element)) <= 2].index, inplace=True)
        self.df["recipe_nutrition_details_formatted"] = self.df["recipe_nutrition_details"].apply(lambda tags: [kv["recipe_nutrition_details"] for kv in eval(tags)])

        self.df["recipe_nutrition_details_formatted"] = self.df["recipe_nutrition_details_formatted"].apply(lambda element: self.recipe_nutrition_formatter(element))


    def clean_df(self, cols: List) -> None:
        # Check if the specified columns exist in the DataFrame
        non_existing_cols = [col for col in cols if col not in self.df.columns]
        
        if non_existing_cols:
            raise ValueError(f"The following columns do not exist in the DataFrame: {non_existing_cols}")
        
        # Drop the specified columns
        self.df.drop(cols, axis=1, inplace=True)
        print("Unwanted columns dropped successfully!")
        



## FLOW
1- create an xlsx reformatter pipeline
  - unnecessary features
  - missing rows (any of the wanted col_names)
  - feature engineering via regex and dict indexing

2- record to json, orient record, indent = 4

In [207]:
df = pd.read_excel(r"C:\Users\ayhan\Desktop\ChefApp\artifacts\recipes\new_data\chineese\chineese_appetizers.xlsx" )
df.head(5)
df["recipe_nutrition_details"].iloc[1]

'[{"recipe_nutrition_details":"98\\nCalories\\n\\n\\n6g \\nFat\\n\\n\\n6g \\nCarbs\\n\\n\\n7g \\nProtein"}]'

In [210]:
df.recipe_details_table.iloc[1]

'[{"recipe_details_table":"Prep Time:\\n10 mins\\n\\n\\nCook Time:\\n 2 hrs 45 mins\\n\\n\\nTotal Time:\\n 2 hrs 55 mins\\n\\n\\nServings:\\n10 \\n\\n\\nYield:\\n10 eggs"}]'

In [330]:
features_to_use = ["card", "card-href", "sub_category", "recipe_name", "recipe_details_table","recipe_ingredients", "recipe_directions", "recipe_tags", "recipe_nutrition_details"]

In [331]:
preprocessor = Preprocessor()
df = preprocessor.read_xlsx(r"C:\Users\ayhan\Desktop\ChefApp\artifacts\recipes\new_data\chineese\chineese_appetizers.xlsx", features_to_use)

preprocessor initialized
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 1 to 29
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   sub_category              29 non-null     object
 1   card                      29 non-null     object
 2   card-href                 29 non-null     object
 3   recipe_name               29 non-null     object
 4   recipe_details_table      29 non-null     object
 5   recipe_ingredients        29 non-null     object
 6   recipe_directions         29 non-null     object
 7   recipe_tags               29 non-null     object
 8   recipe_nutrition_details  29 non-null     object
dtypes: object(9)
memory usage: 2.2+ KB
None


In [ ]:
preprocessor.df_img_url_parser(col_name="card")

In [334]:
preprocessor.recipe_tag_formatter()

In [ ]:
preprocessor.recipe_details_table_formatter()

In [336]:
preprocessor.recipe_ingredients_formatter()

In [337]:
preprocessor.recipe_directions_formatter()

In [ ]:
preprocessor.recipe_nutrition_details_formatter()

In [ ]:
preprocessor.df.recipe_nutrition_details_formatted

In [340]:
unwanted_cols = ["card","recipe_tags", "recipe_details_table", "recipe_ingredients", "recipe_directions", "recipe_nutrition_details","sub_category"]
preprocessor.clean_df(unwanted_cols)

Unwanted columns dropped successfully!


In [341]:
preprocessor.df.columns

Index(['card-href', 'recipe_name', 'image_url_formatted',
       'recipe_tags_formatted', 'recipe_details_formatted',
       'recipe_ingredients_formatted', 'recipe_directions_formatted',
       'recipe_nutrition_details_formatted'],
      dtype='object')

In [342]:
preprocessor.df_to_json()

JSON saved to C:\Users\ayhan\Desktop\ChefApp\artifacts\recipes\new_data\chineese\chineese_appetizers.json


In [205]:
preprocessor.df.head(2)

,card-href,recipe_name,image_url,recipe_tags_formatted,recipe_details_formatted,recipe_ingredients_formatted,recipe_directions_formatted,recipe_nutrition_details_formatted
1,https://www.allrecipes.com/recipe/160400/tea-l...,Tea Leaf Eggs,https://www.allrecipes.com/thmb/CChB81V-kLe5Dz...,"[Recipes, Appetizers and Snacks, chineese_appe...","{'Prep Time': None, 'Cook Time': None, 'Additi...","[1 tablespoon black tea leaves, 2 (3 inch) cin...","[Place the tea, cinnamon, star anise, five-spi...","{'calories': '98', 'fat': '6g', 'carbs': '6g',..."
2,https://www.allrecipes.com/recipe/284414/golde...,Golden Sesame Balls,https://www.allrecipes.com/thmb/odHYwlfyyl2w65...,"[Recipes, Cuisine, Asian, Chinese, chineese_ap...","{'Prep Time': None, 'Cook Time': None, 'Additi...","[1 ounce white sugar, ⅓ ounce shortening, 3 ou...",[Combine sugar and shortening in a bowl to mak...,"{'calories': '217', 'fat': '15g', 'carbs': '19..."


In [7]:
features = df.columns

In [8]:
features = ['sub_category', 'card',
       'card-href', 'recipe_name', 'recipe_details_table',
       'recipe_ingredients', 'recipe_directions', 'recipe_tags',
       'recipe_nutrition_details']

Index(['web-scraper-order', 'web-scraper-start-url', 'sub_category', 'card',
       'card-href', 'recipe_name', 'recipe_details_table',
       'recipe_ingredients', 'recipe_directions', 'recipe_tags',
       'recipe_nutrition_details'],
      dtype='object')

In [164]:
#sub_tag = df.loc[0, "sub_category"]
sub_tag
df["sub_category"] = sub_tag

In [ ]:
df["recipe_tags"][1:-1].map(lambda tags: eval(tags))

In [26]:
[kv["recipe_tags"]  for kv   in eval(df["recipe_tags"][1])]

['Recipes', 'Everyday Cooking', 'Vegan']

In [31]:
df.drop(df[df["recipe_tags"].isnull()].index, inplace=True)


In [34]:

df["recipe_tags_formatted"] = df["recipe_tags"].apply(lambda tags: [kv["recipe_tags"] for kv in eval(tags)])


In [66]:
df.drop(df[~df["recipe_details_table"].str.contains("Time")].index, inplace=True)

In [37]:
df["recipe_details_formatted"] = df["recipe_details_table"].apply(lambda tags: [kv["recipe_details_table"] for kv in eval(tags)])

In [71]:
df.drop(df[df["recipe_ingredients"].apply(lambda element: len(element))<=2].index, inplace=True)

In [74]:
df["recipe_ingredients_formatted"] = df["recipe_ingredients"].apply(lambda tags: [kv["recipe_ingredients"] for kv in eval(tags)])


In [91]:
from typing import List
def recipe_ingredients_formatter(ingredients:List)->List:
  """assumes the list has 1 element, if it contains multiple elements, then just ' '.join()  them """
  ingredient_str = ingredients[0]
  ingredients_list = ingredient_str.split('\n')

  # Remove empty strings and strip any leading/trailing whitespace
  ingredients_list = [ingredient.strip() for ingredient in ingredients_list if ingredient.strip()]
  return ingredients_list


In [92]:
recipe_ingredients_formatter(df["recipe_ingredients_formatted"][7])

['2 tablespoons vegetable oil',
 '½ cup diced sweet onion',
 '1 tablespoon grated ginger root',
 '1 clove garlic, minced',
 '1 ½ cups fresh pineapple chunks',
 '½  diced red bell pepper',
 '⅓ cup thawed frozen peas',
 '2 cups cold, cooked white rice',
 '2 tablespoons low-sodium soy sauce, or to taste',
 '1 teaspoon sesame oil',
 '2 tablespoons sliced green onion',
 '1 teaspoon toasted sesame seeds']

In [97]:
#formatting the whole column
df["recipe_ingredients_formatted"] = df["recipe_ingredients_formatted"].apply(lambda ingredient_list: recipe_ingredients_formatter(ingredient_list))


In [100]:
df.drop(df[df["recipe_directions"].apply(lambda element: len(element))<=2].index, inplace=True)

In [102]:
df["recipe_directions_formatted"] = df["recipe_directions"].apply(lambda tags: [kv["recipe_directions"] for kv in eval(tags)])


In [108]:
len(df["recipe_directions_formatted"][2]), df["recipe_directions_formatted"][2]

(6,
 ['Combine frozen spinach and water in a 1-quart microwave-safe casserole dish. Cover and microwave on high for 5 minutes, stirring halfway through the cooking time. Break up any clumps with a fork and cook for 1 more minute. Transfer to a colander to drain. When cool enough to handle, squeeze small handfuls of spinach to release remaining water and place spinach in a large mixing bowl.',
  'Add Beyond Beef, nutritional yeast, bread crumbs, tomato paste, minced garlic, oregano, basil, onion powder, salt, and pepper to the spinach. Mix with your hands until all ingredients are thoroughly combined. Roll into 1 1/4-inch diameter meatballs.',
  'Preheat oven to 400 degrees F (200 degrees C). Line a large rimmed baking sheet with parchment paper.',
  'Heat 1 tablespoon olive oil over medium heat in a large Dutch oven. Working in batches, add about 12 meatballs and cook, turning often and being careful not to burn, until browned on all sides, about 5 to 7 minutes. Transfer meatballs to t

In [110]:
df.drop(df[df["recipe_nutrition_details"].apply(lambda element: len(element))<=2].index, inplace=True)

In [112]:
df["recipe_nutrition_details_formatted"] = df["recipe_nutrition_details"].apply(lambda tags: [kv["recipe_nutrition_details"] for kv in eval(tags)])


In [135]:
df["recipe_nutrition_details_formatted"].iloc[0]

['350', 'Calories', '7g', 'Fat', '57g', 'Carbs', '16g', 'Protein']

In [153]:
def recipe_nutrition_formatter(nutrition_details:List)->List:
  nutrition_info = {
      "calories":0,
      "fat" : 0,
      "carbs":0,
      "protein":0}

  for index, item in enumerate(nutrition_details):
    if item.lower() in nutrition_info:
      nutrition_info[item.lower()] = nutrition_details[index-1]

  return  nutrition_info

In [154]:
recipe_nutrition_formatter(df["recipe_nutrition_details_formatted"].iloc[0])

{'calories': '350', 'fat': '7g', 'carbs': '57g', 'protein': '16g'}

In [ ]:
df["recipe_nutrition_details_formatted"] = df["recipe_nutrition_details_formatted"].apply( lambda element:  recipe_nutrition_formatter(element))



In [171]:
df.head(5).iloc[:,2:]

,sub_category,card,card-href,recipe_name,recipe_details_table,recipe_ingredients,recipe_directions,recipe_tags,recipe_nutrition_details,recipe_tags_formatted,recipe_details_formatted,recipe_ingredients_formatted,recipe_directions_formatted,recipe_nutrition_details_formatted
1,Vegan Main Dishes,"<img\nsrc=""https://www.allrecipes.com/thmb/kFt...",https://www.allrecipes.com/recipe/245424/mujad...,Mujadarra (Lentils with Rice),"[{""recipe_details_table"":""Prep Time:\n15 mins\...","[{""recipe_ingredients"":""14 ounces dry brown le...","[{""recipe_directions"":""Fill a large pot with l...","[{""recipe_tags"":""Recipes""},{""recipe_tags"":""Eve...","[{""recipe_nutrition_details"":""350\nCalories\n\...","[Recipes, Everyday Cooking, Vegan, Vegan Main ...",[Prep Time:\n15 mins\n\n\nCook Time:\n30 mins\...,"[14 ounces dry brown lentils, 2 carrots, grat...",[Fill a large pot with lightly salted water; a...,"{'calories': '350', 'fat': '7g', 'carbs': '57g..."
2,Vegan Main Dishes,"<img\nsrc=""https://www.allrecipes.com/thmb/W9m...",https://www.allrecipes.com/recipe/278267/beyon...,Beyond Beef Vegan Meatballs,"[{""recipe_details_table"":""Prep Time:\n25 mins\...","[{""recipe_ingredients"":""1 (10 ounce) package f...","[{""recipe_directions"":""Combine frozen spinach ...","[{""recipe_tags"":""Recipes""},{""recipe_tags"":""Mai...","[{""recipe_nutrition_details"":""411\nCalories\n\...","[Recipes, Main Dishes, Meatball Recipes, Vegan...",[Prep Time:\n25 mins\n\n\nCook Time:\n 1 hr\n\...,"[1 (10 ounce) package frozen chopped spinach, ...",[Combine frozen spinach and water in a 1-quart...,"{'calories': '411', 'fat': '26g', 'carbs': '19..."
3,Vegan Main Dishes,"<img\nsrc=""https://www.allrecipes.com/thmb/8hE...",https://www.allrecipes.com/recipe/263128/jicam...,Jicama Tortillas,"[{""recipe_details_table"":""Prep Time:\n15 mins\...","[{""recipe_ingredients"":""1 large jicama""}]","[{""recipe_directions"":""Wash jicama thoroughly ...","[{""recipe_tags"":""Recipes""},{""recipe_tags"":""Cui...","[{""recipe_nutrition_details"":""25\nCalories\n\n...","[Recipes, Cuisine, Latin American, Mexican, Ve...",[Prep Time:\n15 mins\n\n\nTotal Time:\n15 mins...,[1 large jicama],[Wash jicama thoroughly with cool water and re...,"{'calories': '25', 'fat': '0g', 'carbs': '6g',..."
4,Vegan Main Dishes,"<img\nsrc=""https://www.allrecipes.com/thmb/xvp...",https://www.allrecipes.com/recipe/232210/cousc...,Couscous with Olives and Sun-Dried Tomato,"[{""recipe_details_table"":""Prep Time:\n20 mins\...","[{""recipe_ingredients"":""1 ¼ cups vegetable bro...","[{""recipe_directions"":""Bring 1 1/4 cup vegetab...","[{""recipe_tags"":""Cuisine""},{""recipe_tags"":""Afr...","[{""recipe_nutrition_details"":""528\nCalories\n\...","[Cuisine, African, North African, Egyptian, Ve...",[Prep Time:\n20 mins\n\n\nCook Time:\n30 mins\...,"[1 ¼ cups vegetable broth, 1 ¼ cups water, 2 c...",[Bring 1 1/4 cup vegetable broth and water to ...,"{'calories': '528', 'fat': '29g', 'carbs': '56..."
5,Vegan Main Dishes,"<img\nsrc=""https://www.allrecipes.com/thmb/0ZE...",https://www.allrecipes.com/recipe/25331/americ...,American-Style Red Beans and Rice,"[{""recipe_details_table"":""Prep Time:\n5 mins\n...","[{""recipe_ingredients"":""1 tablespoon olive oil...","[{""recipe_directions"":""Combine olive oil, kidn...","[{""recipe_tags"":""Recipes""},{""recipe_tags"":""Mai...","[{""recipe_nutrition_details"":""511\nCalories\n\...","[Recipes, Main Dishes, Rice, Beans and Rice Re...",[Prep Time:\n5 mins\n\n\nCook Time:\n30 mins\n...,"[1 tablespoon olive oil, 1 (15 ounce) can kidn...","[Combine olive oil, kidney beans, tomato sauce...","{'calories': '511', 'fat': '5g', 'carbs': '101..."


In [ ]:
df["recipe_tags_formatted"].iloc[1:].apply(lambda element: element.append("Vegan Main Dishes"))